## Bentley auth using native, SPA or web app credentials

### Option 1
#### Automatic auth using the Evo Python SDK

In [ ]:
# Define your client ID and callback URL.

client_id = "<your-client-id>"
redirect_url = "<your-redirect-url>"

In [ ]:
from evo.notebooks import ServiceManagerWidget

cache_location = "cache"

manager = await ServiceManagerWidget.with_auth_code(
    redirect_url=redirect_url,
    client_id=client_id,
    cache_location=cache_location,
).login()

In [ ]:
environment = manager.get_environment()
print(f"Hub URL: {environment.hub_url}")
print(f"Org ID: {environment.org_id}")
print(f"Workspace ID: {environment.workspace_id}")

### Option 2
#### Manual auth using common Python packages

In [ ]:
# Define your client ID, callback URL and Evo scopes.
# NOTE: Add the scope "offline_access" to the list if you require a refresh token.

client_id = "<your-client-id>"
redirect_url = "<your-redirect-url>"
scope = ["evo.workspace", "evo.discovery", "evo.object", "evo.blocksync", "evo.file"]

base_uri = "https://ims.bentley.com"

In [ ]:
import datetime
import webbrowser
from http.server import BaseHTTPRequestHandler, HTTPServer
from urllib.parse import urlparse

import jwt
from authlib.common.security import generate_token
from authlib.integrations.requests_client import OAuth2Session


def get_access_token(
    client_id: str,
    redirect_url: str,
    authorisation_url: str,
    access_token_url: str,
    scope: str,
) -> str:
    """Get an access token."""

    port = urlparse(redirect_url).port

    class OAuthHttpServer(HTTPServer):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            self.authorization_code = ""

    class OAuthHttpHandler(BaseHTTPRequestHandler):
        def do_GET(self):
            self.send_response(200)
            self.send_header("Content-Type", "text/html")
            self.end_headers()
            self.wfile.write("Redirecting to the Bentley login page\n".encode("UTF-8"))

            client.fetch_token(
                url=access_token_url,
                authorization_response=self.path,
                state=state,
                code_verifier=code_verifier,
                grant_type="authorization_code",
            )

            self.wfile.write(
                """
                <html>
                  <body>
                    <h2>Authorization request to Bentley has been completed.</h1>
                    <h3>You may now close this tab or window now.</h3>
                  </body>
                </html>
                """.encode("UTF-8")
            )
            self.wfile.write(
                '<script> setTimeout("window.close()", 2500);</script>'.encode("UTF-8")
            )  # Timeout only works if already logged in

    with OAuthHttpServer(("", port), OAuthHttpHandler) as httpd:
        client = OAuth2Session(
            client_id=client_id,
            scope=scope,
            redirect_uri=redirect_url,
            code_challenge_method="S256",
        )

        code_verifier = generate_token(48)
        auth_uri, state = client.create_authorization_url(authorisation_url, code_verifier=code_verifier)
        webbrowser.open_new(auth_uri)
        httpd.handle_request()

    return client.token["access_token"]


token = get_access_token(
    client_id=client_id,
    redirect_url=redirect_url,
    authorisation_url=f"{base_uri}/connect/authorize",
    access_token_url=f"{base_uri}/connect/token",
    scope=scope,
)

print("Access token:")
print(token)

decoded = jwt.decode(token, options={"verify_signature": False}, algorithms=["RS256"])
exp_timestamp = decoded["exp"]
exp_datetime = datetime.datetime.fromtimestamp(exp_timestamp, datetime.timezone.utc)
print(f"Token expires at:\n{exp_datetime} UTC")